In [27]:
import pandas as pd
import numpy as np
import requests
import datetime
import json
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode() 
import Quandl

py.sign_in('enrico.tesio','ttv7w7heht')

# Get data FROM U.S. Energy Information Administration using the below API key
api_key = '508B0FE56B9D2507A2CE82A700EE86D0'
req = 'http://api.eia.gov/series/?api_key={}&series_id=NG.N3010{}2.M'.format(api_key,'US')
r_us = requests.get(req)
data_us = json.loads(r_us.text)
dates = []
volumes = []
# set index using the total U.S. data
for element in data_us['series'][0]['data']:
    dates.append(datetime.datetime.strptime(element[0], '%Y%m').date())
    volumes.append(float(element[1]))    
df_us = pd.DataFrame(index=dates, columns=['US Consumption'])
df_us['US Consumption'] = volumes
df_us.sort_index(ascending=True, inplace=True)
df_us.index = pd.to_datetime(df_us.index)
df_us.index.name = 'Date'
# download henry hub settlements from Quandl
henry_hub = Quandl.get('FRED/GASPRICE')

conversion = 0.9756 # MMBTU to MMcft
df = pd.merge(df_us, henry_hub, how='outer', left_index=True, right_index=True).dropna(how='any')*conversion
df.columns = ['US Consumption', 'HH']
#df = df.pct_change()

# plot data
trace_1 = go.Scatter(x=df.index, y=df['US Consumption'], name='U.S. Consumption', yaxis='y1')
trace_2 = go.Scatter(x=df.index, y=df['HH'], name='Henry Hub', yaxis='y2')
data = [trace_1, trace_2]
layout = dict(title='U.S. consumption and Henry Hub price (1993 - Today)',
             xaxis=dict(title='Time'),
             yaxis=dict(title='US gas consumption (MMcft)'),
             yaxis2=dict(title='Henry Hub price ($/MMcft)', side='right',
                         titlefont=dict(color='rgb(148, 103, 189)'),tickfont=dict(color='rgb(148, 103, 189)'),
                         overlaying='y'),
             legend=dict(x=0.75,y=1))
fig = dict(data=data, layout=layout)
iplot(fig, show_link=False)

df_ret = df.pct_change()
trace_1 = go.Scatter(x=df_ret.index, y=df_ret['US Consumption'], name='U.S. Consumption', yaxis='y1')
trace_2 = go.Scatter(x=df_ret.index, y=df_ret['HH'], name='Henry Hub', yaxis='y2')
data = [trace_1, trace_2]
layout = dict(title='U.S. consumption and Henry Hub price % variations (1993 - Today)',
             xaxis=dict(title='Time'),
             yaxis=dict(title='US gas consumption variation'),
             yaxis2=dict(title='Henry Hub return', side='right',
                         titlefont=dict(color='rgb(148, 103, 189)'),tickfont=dict(color='rgb(148, 103, 189)'),
                         overlaying='y'),
             legend=dict(x=0.75,y=1))
fig = dict(data=data, layout=layout)
iplot(fig, show_link=False)

correlation_matrix = df_ret.corr()
correlation = correlation_matrix.ix[0,1]
print('Correlation between monthly variations in gas consumption and Henry Hub monthly returns: {}%'.format(round(correlation*100,1)))

Correlation between monthly variations in gas consumption and Henry Hub monthly returns: 22.3%


In [56]:
start_date = datetime.datetime(2016,5,1)
end_date = datetime.datetime(2017,3,31)
dates = pd.date_range(start_date, end_date, freq='D')
months = pd.date_range(start_date, end_date, freq='MS')

# mapping between delivery months and time series name 
#(see https://www.quandl.com/collections/futures/cme-natural-gas-futures)
mapping = {
    1:'F', 2:'G', 3:'H', 4:'J',
    5:'K', 6:'M', 7:'N', 8:'Q', 
    9:'U', 10:'V', 11:'X', 12:'Z'
}
forward_curve = pd.DataFrame(index=months, columns=['Henry Hub'])
for m in months:
    year = str(m.year)
    letter = mapping[m.month]
    series_name = 'CME/NG{0}{1}'.format(letter, year)
    temp = Quandl.get(series_name)
    value = temp.loc[temp.index==temp.last_valid_index()]['Settle'].values[0]*conversion
    forward_curve.ix[m]['Henry Hub'] = value
    
fc_data = [go.Scatter(x=forward_curve.index, y=forward_curve['Henry Hub'], name='HH Forward Curve')]
fc_layout = dict(title='Henry Hub forward curve',
                xaxis=dict(title='Delivery Month'),
                yaxis=dict(title='Forward Price ($/MMcft)'))
fc_fig = dict(data=fc_data, layout=fc_layout)
iplot(fc_fig, show_link=False)

# TODO: Clelow-Stricland single factor model

ErrorDownloading: Error Downloading! HTTP Error 429: 